## **<center>D212 OFM3 TASK 3: ASSOCIATION RULES AND LIFT ANALYSIS<center>** 

Julia Amanda Terzin  
ID:009968680  
Mentor: Linda Howel   
1-385-428-2729    
linda.howell@wgu.edu   
August 18, 2022

### Part I: Research Question

#### A. Describe the purpose of this data mining report by doing the following:

#### A.1.  Propose one question relevant to a real-world organizational situation that you will answer using market basket analysis.

Can we use market basket analysis (MBA)to identify what items are frequently purchased together by customers?

#### A.2. Define one goal of the data analysis. Ensure that your goal is reasonable within the scope of the scenario and is represented in the available data.


The goal of this analysis is to identify what items are commonly purchased together.  The information will be utilized by the Telco company to understand their customers purchasing behavior to inform their business decisions.

### Part II:  Market Basket Justification

#### B.  Explain the reasons for using market basket analysis by doing the following:

#### B.1.  Explain how market basket analyzes the selected dataset. Include expected outcomes.

MBA evaluates a database of transactions and computes the likelihood of purchasing an item (consequent) given the purchase of one or more other items (antecedent). After preparing the data by transforming it into a DataFrame with the items as columns, the individual transactions as rows, and the values as Boolean (true or false),  mlxtend.frequent_patterns apriori will be used to generate a data frame of frequent sets of items purchased together. The frequent item sets are then put into mlxtend.frequent_patterns association_rules to obtain support, confidence, and lift.  (Pipis, 2020) These metrics measure the association between the purchase of the antecedent and the consequent. Further discussion of the metrics can be found in section D.1.  

The expected outcome is a set of rules listing antecedents and consequents with their respective support, confidence, and lift. 

#### B.2.  Provide one example of transactions in the dataset.

The first transaction in the data set is:

![image info](transaction.png)

The same transaction in list form is: 

![image info](transaction_b.png)

#### B.3. Summarize one assumption of market basket analysis.

One assumption of MBA is that two or more items frequently purchased together imply an association that the purchase of one or more specific items leads to the purchase of another specific item. (Kamakura, 2012)

### Part III: Data Preparation and Analysis

#### C.  Prepare and perform market basket analysis by doing the following:

#### C.1.  Transform the dataset to make it suitable for market basket analysis. Include a copy of the cleaned dataset.

##### Import Packages

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, apriori
import warnings

In [2]:
## Versions
print(pd.__version__)
print(np.__version__)

1.2.5
1.20.1


In [3]:
warnings.filterwarnings('ignore')

##### Load Data

In [4]:
df = pd.read_csv('c:/Users/jater/Downloads/D205/teleco_market_basket.csv')

In [5]:
df_copy = df.copy(deep=True)

##### Explore Data

In [6]:
df.shape

(15002, 20)

In [7]:
df.info() #Column names, data types, non-nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [8]:
df.isnull().sum() 

Item01     7501
Item02     9255
Item03    10613
Item04    11657
Item05    12473
Item06    13138
Item07    13633
Item08    14021
Item09    14348
Item10    14607
Item11    14746
Item12    14848
Item13    14915
Item14    14955
Item15    14977
Item16    14994
Item17    14998
Item18    14998
Item19    14999
Item20    15001
dtype: int64

In [9]:
df.isnull().all(axis=0).sum()

0

In [10]:
df.isnull().all(axis=1).sum()

7501

In [11]:
# Drop empty rows
df = df.dropna(how='all')
df.isnull().all(axis=1).sum()

0

In [12]:
df.duplicated().sum()

2325

In [13]:
pd.set_option('display.max_columns', None)
df.head()

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,UNEN Mfi Certified 5-pack Lightning Cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cat8 Ethernet Cable,HP 65 ink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Dust-Off Compressed Gas 2 pack,Screen Mom Screen Cleaner kit,Moread HDMI to VGA Adapter,HP 62XL Tri-Color ink,Apple USB-C Charger cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.reset_index(drop=True)

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
1,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UNEN Mfi Certified 5-pack Lightning Cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cat8 Ethernet Cable,HP 65 ink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dust-Off Compressed Gas 2 pack,Screen Mom Screen Cleaner kit,Moread HDMI to VGA Adapter,HP 62XL Tri-Color ink,Apple USB-C Charger cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,SanDisk 32GB Ultra SDHC card,Vsco 70 pack stickers,SanDisk 128GB microSDXC card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,Apple Lightning to Digital AV Adapter,Nylon Braided Lightning to USB cable,Apple Pencil,USB 2.0 Printer cable,ARRIS SURFboard SB8200 Cable Modem,Apple USB-C Charger cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,Falcon Dust Off Compressed Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,HP 63XL Ink,Apple USB-C Charger cable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Preparing Transactions

In [15]:
df.shape

(7501, 20)

In [16]:
#https://towardsdatascience.com/apriori-algorithm-for-association-rule-learning-how-to-find-clear-links-between-transactions-bf7ebc22cf0a
# Create a list of lists from a dataframe
transactions=df.stack().groupby(level=0).apply(list).tolist()
# Show what it looks like
transactions[0:5]

[['Logitech M510 Wireless mouse',
  'HP 63 Ink',
  'HP 65 ink',
  'nonda USB C to USB Adapter',
  '10ft iPHone Charger Cable',
  'HP 902XL ink',
  'Creative Pebble 2.0 Speakers',
  'Cleaning Gel Universal Dust Cleaner',
  'Micro Center 32GB Memory card',
  'YUNSONG 3pack 6ft Nylon Lightning Cable',
  'TopMate C5 Laptop Cooler pad',
  'Apple USB-C Charger cable',
  'HyperX Cloud Stinger Headset',
  'TONOR USB Gaming Microphone',
  'Dust-Off Compressed Gas 2 pack',
  '3A USB Type C Cable 3 pack 6FT',
  'HOVAMP iPhone charger',
  'SanDisk Ultra 128GB card',
  'FEEL2NICE 5 pack 10ft Lighning cable',
  'FEIYOLD Blue light Blocking Glasses'],
 ['Apple Lightning to Digital AV Adapter',
  'TP-Link AC1750 Smart WiFi Router',
  'Apple Pencil'],
 ['UNEN Mfi Certified 5-pack Lightning Cable'],
 ['Cat8 Ethernet Cable', 'HP 65 ink'],
 ['Dust-Off Compressed Gas 2 pack',
  'Screen Mom Screen Cleaner kit',
  'Moread HDMI to VGA Adapter',
  'HP 62XL Tri-Color ink',
  'Apple USB-C Charger cable']]

In [17]:
# Instantiate transaction encoder 
encoder = TransactionEncoder().fit(transactions)
# One-hot encode itemsets by applying fit and transform 
onehot = encoder.transform(transactions) 
# Convert one-hot encoded data to DataFrame 
onehot = pd.DataFrame(onehot, columns = encoder.columns_) 
onehot

,10ft iPHone Charger Cable,10ft iPHone Charger Cable 2 Pack,3 pack Nylon Braided Lightning Cable,3A USB Type C Cable 3 pack 6FT,5pack Nylon Braided USB C cables,ARRIS SURFboard SB8200 Cable Modem,Anker 2-in-1 USB Card Reader,Anker 4-port USB hub,Anker USB C to HDMI Adapter,Apple Lightning to Digital AV Adapter,Apple Lightning to USB cable,Apple Magic Mouse 2,Apple Pencil,Apple Pencil 2nd Gen,Apple Power Adapter Extension Cable,Apple USB-C Charger cable,AutoFocus 1080p Webcam,BENGOO G90000 headset,Blue Light Blocking Glasses,Blue Light Blocking Glasses 2pack,Brother Genuine High Yield Toner Cartridge,Cat 6 Ethernet Cable 50ft,Cat8 Ethernet Cable,CicTsing MM057 2.4G Wireless Mouse,Cleaning Gel Universal Dust Cleaner,Creative Pebble 2.0 Speakers,DisplayPort ot HDMI adapter,Dust-Off Compressed Gas,Dust-Off Compressed Gas 2 pack,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses,Falcon Dust Off Compressed Gas,HOVAMP Mfi 6pack Lightning Cable,HOVAMP iPhone charger,HP 61 2 pack ink,HP 61 Tri-color ink,HP 61 ink,HP 62XL Tri-Color ink,HP 62XL ink,HP 63 Ink,HP 63 Tri-color ink,HP 63XL Ink,HP 63XL Tri-color ink,HP 64 Tri-Color ink,HP 64 ink,HP 65 ink,HP 902XL ink,HP 952 ink,HP ENVY 5055 printer,HP952XL ink,HooToo USB C Hub,HyperX Cloud Stinger Headset,Jelly Comb 2.4G Slim Wireless mouse,Leader Desk Pad Protector,Logitech M510 Wireless mouse,Logitech MK270 Wireless Keyboard/Mouse,Logitech MK345 Wireless combo,Logitech USB H390 headset,M.2 Screw kit,Mfi-Certified Lightning to USB A Cable,Micro Center 32GB Memory card,Microsot Surface Dock 2,Moread HDMI to VGA Adapter,Mpow HC6 USB Headset,NETGEAR CM500 Cable Modem,NETGEAR Nighthawk WiFi Router,NETGEAR Orbi Home Mesh WiFi System,Nylon Braided Lightning to USB cable,PS4 Headset,Premium Nylon USB Cable,RUNMUS Gaming Headset,SAMSUNG 128GB card,SAMSUNG 256 GB card,SAMSUNG EVO 32GB card,SAMSUNG EVO 64GB card,Sabrent 4-port USB 3.0 hub,SanDisk 128GB Ultra microSDXC card,SanDisk 128GB card,SanDisk 128GB microSDXC card,SanDisk 32GB Ultra SDHC card,SanDisk 32GB card,SanDisk Extreme 128GB card,SanDisk Extreme 256GB card,SanDisk Extreme 32GB 2pack card,SanDisk Extreme Pro 128GB card,SanDisk Extreme Pro 64GB card,SanDisk Ultra 128GB card,SanDisk Ultra 256GB card,SanDisk Ultra 400GB card,SanDisk Ultra 64GB card,Screen Mom Screen Cleaner kit,Stylus Pen for iPad,Syntech USB C to USB Adapter,TONOR USB Gaming Microphone,TP-Link AC1750 Smart WiFi Router,TP-Link AC4000 WiFi router,TopMate C5 Laptop Cooler pad,UNEN Mfi Certified 5-pack Lightning Cable,USB 2.0 Printer cable,USB C to USB Male Adapter,USB Type C Cable,USB Type C to USB-A Charger cable,VIVO Dual LCD Monitor Desk mount,VicTsing Mouse Pad,VicTsing Wireless mouse,Vsco 70 pack stickers,Webcam with Microphone,XPOWER A-2 Air Pump blower,YUNSONG 3pack 6ft Nylon Lightning Cable,hP 65 Tri-color ink,iFixit Pro Tech Toolkit,iPhone 11 case,iPhone 12 Charger cable,iPhone 12 Pro case,iPhone 12 case,iPhone Charger Cable Anker 6ft,iPhone SE case,nonda USB C to USB Adapter,seenda Wireless mouse
0,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [18]:
onehot.to_csv('D212Task3clean.csv')

#### C.2. Execute the code used to generate association rules with the Apriori algorithm. Provide screenshots that demonstrate the error-free functionality of the code.

In [19]:
# Compute frequent itemsets 
frequent_itemsets = apriori(onehot, min_support = 0.003, use_colnames = True) 
# Print number of itemsets 
print(len(frequent_itemsets))

1443


In [20]:
# Compute association rules for support >0.009 to start with the top 0.9% of the itemsets
rules = association_rules(frequent_itemsets, metric = 'support' , min_threshold = 0.009)
rules = rules[['antecedents','consequents','support','confidence','lift']]
# Print number of rules 
print(len(rules))

554


In [21]:
# Prune rules for confidence > 0.4 & lift > 2 for the rules with the most significance
filtered_rules = rules[(rules['confidence'] > 0.4) & (rules['lift'] > 2)]
print(len(filtered_rules))

6


#### C.3.  Provide values for the support, lift, and confidence of the association rules table.

In [22]:
pd.set_option("max_colwidth", None)

In [23]:
filtered_rules

,antecedents,consequents,support,confidence,lift
405,"(USB 2.0 Printer cable, Apple Lightning to Digital AV Adapter)",(Apple Pencil),0.009065,0.412121,2.293265
423,"(SanDisk Ultra 64GB card, Apple Pencil)",(Dust-Off Compressed Gas 2 pack),0.010132,0.506667,2.125563
488,"(SanDisk Ultra 64GB card, Nylon Braided Lightning to USB cable)",(Dust-Off Compressed Gas 2 pack),0.009199,0.543307,2.279277
506,"(SanDisk Ultra 64GB card, Screen Mom Screen Cleaner kit)",(Dust-Off Compressed Gas 2 pack),0.011065,0.503030,2.110308
513,"(SanDisk Ultra 64GB card, Dust-Off Compressed Gas 2 pack)",(VIVO Dual LCD Monitor Desk mount),0.017064,0.416938,2.394681
549,"(SanDisk Ultra 64GB card, Screen Mom Screen Cleaner kit)",(VIVO Dual LCD Monitor Desk mount),0.009732,0.442424,2.541060


#### C.4. Identify the top three rules generated by the Apriori algorithm. Include a screenshot of the top rules along with their summaries.

In [24]:
filtered_rules.sort_values(by = 'support', ascending = False).head(3)

,antecedents,consequents,support,confidence,lift
513,"(SanDisk Ultra 64GB card, Dust-Off Compressed Gas 2 pack)",(VIVO Dual LCD Monitor Desk mount),0.017064,0.416938,2.394681
506,"(SanDisk Ultra 64GB card, Screen Mom Screen Cleaner kit)",(Dust-Off Compressed Gas 2 pack),0.011065,0.503030,2.110308
423,"(SanDisk Ultra 64GB card, Apple Pencil)",(Dust-Off Compressed Gas 2 pack),0.010132,0.506667,2.125563


### Part IV:  Data Summary and Implications

#### D. Summarize your data analysis by doing the following:

#### D.1.  Summarize the significance of support, lift, and confidence from the results of the analysis.

1.	**Support** is the proportion of the transactions that contain the itemset. The range is 0-1 with closer to 1 being better. 

**<center>Support(X → Y ) =  # of transactions with X&Y / #  of all transactions</center>**

2.	**Confidence** is the proportion of transactions that contain the antecedent and consequent divided by the proportion of transactions that contain the antecedent.  It measures the likelihood of the association or the percentage of the time that when the antecedent is purchased, the consequent is also purchased. The range is 0-1 with closer to 1 being better.

**<center>Confidence(X → Y) = % of transactions with X&Y /  % of transactions with X</center>**

3.	**Lift** compares the likelihood of an association between items being purchased together versus the random occurrence of the items being purchased. It measures the strength of the association. A lift of 1 means there is no relationship and only occurs together randomly.  > 1 means there is a relationship and the higher the number the more likely the items occur together due to association than randomly. < 1 means there is a negative association, and the items occur together fewer times than they would randomly.

**<center>Lift(X → Y) = Support(X → Y ) / (Support(X) * Support(Y))</center>**

Rules were generated with the minimum criteria of support> 0.009, confidence > 0.4, and lift > 2. In the top three rules above, support of 0.017064, 0.011065, and 0.010132 means that the antecedents and consequents occur together in 1.7%, 1.1%, and 1.0% of the transactions. Confidence of 0.416938, 0.503030, and 0.506667 means that 41.7%, 50.3%, and 50.7% of the times the antecedent was purchased, the consequent was also purchased.  The lift of the top three rules shows that the occurrence of the antecedent and consequent being purchased together are 2.394681, 2.110308, and 2.125563 times more likely than would be expected randomly.(Goel, 2018)

#### D.2.  Discuss the practical significance of the findings from the analysis.

The practical significance of the analysis is that MBA can be used to recommend additional items to customers in online sales based on initial items placed in their cart or for placement of the items together in brick-and-mortar stores. For instance, in the top rule, #3685, customers buying SanDisk Ultra 64GB card and Dust-Off Compressed Gas 2 pack also purchased a VIVO Dual LCD Monitor Desk Mount.  A recommender engine can be implemented in their online sales to recommend the desk mount to customers placing the memory card and gas pack.  

#### D.3. Recommend a course of action for the real-world organizational situation from part A1 based on your results from part D1.

Based on the results of this analysis, for online sales, I recommend the Telco company implement a recommender engine to suggest additional items for purchase based on items they place in their carts. For brick-and-mortar stores, they should place items strategically to boost sales of items commonly purchased together. For sales marketing, they should use the results to bundle items for specials. Lastly, the results should be used for inventory management. High support items should be well stocked. (Optimo, 2022)

### Part V: Attachments

#### E. Provide a Panopto video recording that includes a demonstration of the functionality of the code used forthe analysis and a summary of the programming environment.

https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=1424d30b-54df-4562-9b66-aef5013ee4b9

#### F. Record all web sources used to acquire data or segments of third-party code to support the application. Ensure the web sources are reliable.

https://cgp-oex.wgu.edu/courses/course-v1:WGUx+OEX0026+v02/courseware/1f468770545f494fa657b4dc0ed3762f/2b5f23c5dad64357b352728993788677/6?activate_block_id=block-v1%3AWGUx%2BOEX0026%2Bv02%2Btype%40vertical%2Bblock%403f0422d47d8b4a3eaec25de3bced8bf8

https://towardsdatascience.com/principal-component-analysis-algorithm-in-real-life-discovering-patterns-in-a-real-estate-dataset-18134c57ffe7

#### G. Acknowledge sources, using in-text citations and references, for content that is quoted, paraphrased, or summarized.

Goel, Niharika. 2018. Market Basket Analysis: Association Rules.
https://medium.com/@niharika.goel/market-basket-analysis-association-rules-e7c27b377bd8

Kamakura, Wagner. 2012.  Sequential market basket analysis.
http://wak2.web.rice.edu/bio/My%20Reprints/Sequential%20Market%20Basket%20Analysist.pdf

Pipis, George. 2020. A Tutorial About Market Basket Analysis in Python: A gentle introduction of Market Basket Analysis and its Applications.
https://medium.com/swlh/a-tutorial-about-market-basket-analysis-in-python-predictive-hacks-497dc6e06b27

Optimo. 2022. How Market Basket Analysis Can Skyrocket Your Product Sales.
https://optimoanalytics.com/blog/how-market-basket-analysis-can-skyrocket-your-product-sales/#Where_to_use_market_basket_analysis_data